In [22]:
# @title Imports

import os
from io import BytesIO
import tarfile
import tempfile
from six.moves import urllib

from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image

import tensorflow as tf

import cv2

import time
import datetime

In [2]:
#@title Helper methods


class DeepLabModel(object):
  """Class to load deeplab model and run inference."""

  INPUT_TENSOR_NAME = 'ImageTensor:0'
  OUTPUT_TENSOR_NAME = 'SemanticPredictions:0'
  INPUT_SIZE = 513
  FROZEN_GRAPH_NAME = 'frozen_inference_graph'

  def __init__(self, tarball_path):
    """Creates and loads pretrained deeplab model."""
    self.graph = tf.Graph()

    graph_def = None
    # Extract frozen graph from tar archive.
    tar_file = tarfile.open(tarball_path)
    for tar_info in tar_file.getmembers():
      if self.FROZEN_GRAPH_NAME in os.path.basename(tar_info.name):
        file_handle = tar_file.extractfile(tar_info)
        graph_def = tf.compat.v1.GraphDef.FromString(file_handle.read())
        break

    tar_file.close()

    if graph_def is None:
      raise RuntimeError('Cannot find inference graph in tar archive.')

    with self.graph.as_default():
      tf.import_graph_def(graph_def, name='')

    self.sess = tf.compat.v1.Session(graph=self.graph)

  def run(self, image):
    """Runs inference on a single image.

    Args:
      image: A PIL.Image object, raw input image.

    Returns:
      resized_image: RGB image resized from original input image.
      seg_map: Segmentation map of `resized_image`.
    """
    width, height = image.size
    resize_ratio = 1.0 * self.INPUT_SIZE / max(width, height)
    target_size = (int(resize_ratio * width), int(resize_ratio * height))
    resized_image = image.convert('RGB').resize(target_size, Image.ANTIALIAS)
    batch_seg_map = self.sess.run(
        self.OUTPUT_TENSOR_NAME,
        feed_dict={self.INPUT_TENSOR_NAME: [np.asarray(resized_image)]})
    seg_map = batch_seg_map[0]
    return resized_image, seg_map


def create_pascal_label_colormap():
  """Creates a label colormap used in PASCAL VOC segmentation benchmark.

  Returns:
    A Colormap for visualizing segmentation results.
  """
  colormap = np.zeros((256, 3), dtype=int)
  ind = np.arange(256, dtype=int)

  for shift in reversed(range(8)):
    for channel in range(3):
      colormap[:, channel] |= ((ind >> channel) & 1) << shift
    ind >>= 3

  return colormap


def label_to_color_image(label):
  """Adds color defined by the dataset colormap to the label.

  Args:
    label: A 2D array with integer type, storing the segmentation label.

  Returns:
    result: A 2D array with floating type. The element of the array
      is the color indexed by the corresponding element in the input label
      to the PASCAL color map.

  Raises:
    ValueError: If label is not of rank 2 or its value is larger than color
      map maximum entry.
  """
  if label.ndim != 2:
    raise ValueError('Expect 2-D input label')

  colormap = create_pascal_label_colormap()

  if np.max(label) >= len(colormap):
    raise ValueError('label value too large.')

  return colormap[label]


def vis_segmentation(image, seg_map):
  """Visualizes input image, segmentation map and overlay view."""
  plt.figure(figsize=(15, 5))
  grid_spec = gridspec.GridSpec(1, 4, width_ratios=[6, 6, 6, 1])

  plt.subplot(grid_spec[0])
  plt.imshow(image)
  plt.axis('off')
  plt.title('input image')

  plt.subplot(grid_spec[1])
  seg_image = label_to_color_image(seg_map).astype(np.uint8)
  plt.imshow(seg_image)
  plt.axis('off')
  plt.title('segmentation map')

  plt.subplot(grid_spec[2])
  plt.imshow(image)
  plt.imshow(seg_image, alpha=0.7)
  plt.axis('off')
  plt.title('segmentation overlay')

  unique_labels = np.unique(seg_map)
  ax = plt.subplot(grid_spec[3])
  plt.imshow(
      FULL_COLOR_MAP[unique_labels].astype(np.uint8), interpolation='nearest')
  ax.yaxis.tick_right()
  plt.yticks(range(len(unique_labels)), LABEL_NAMES[unique_labels])
  plt.xticks([], [])
  ax.tick_params(width=0.0)
  plt.grid('off')
  plt.show()

# cityscape labels
LABEL_NAMES = np.asarray([
    'road','sidewalk','building',
    'wall','fence','pole','traffic light','traffic sign',
    'vegetation','terrain','sky','person','rider','car',
    'truck','bus','train','motorcycle','bicycle','unlabeled',
])

FULL_LABEL_MAP = np.arange(len(LABEL_NAMES)).reshape(len(LABEL_NAMES), 1)
FULL_COLOR_MAP = label_to_color_image(FULL_LABEL_MAP)

In [3]:
FULL_LABEL_MAP

array([[ 0],
       [ 1],
       [ 2],
       [ 3],
       [ 4],
       [ 5],
       [ 6],
       [ 7],
       [ 8],
       [ 9],
       [10],
       [11],
       [12],
       [13],
       [14],
       [15],
       [16],
       [17],
       [18],
       [19]])

In [4]:
#@title Select and download models {display-mode: "form"}

# MODEL_NAME = 'mobilenetv2_coco_cityscapes_train' # 23MB
MODEL_NAME = 'deeplabv3_cityscapes_train' # 439MB
# MODEL_NAME = 'deeplab_cityscapes_xception71_trainfine' # 445MB
# MODEL_NAME = 'deeplab_cityscapes_xception71_trainvalfine' # 446MB

_DOWNLOAD_URL_PREFIX = 'http://download.tensorflow.org/models/'
_MODEL_URLS = {
    'mobilenetv2_coco_cityscapes_train':
        'deeplabv3_mnv2_cityscapes_train_2018_02_05.tar.gz',
    'deeplabv3_cityscapes_train':
        'deeplabv3_cityscapes_train_2018_02_06.tar.gz',
    'deeplab_cityscapes_xception71_trainfine':
        'deeplab_cityscapes_xception71_trainfine_2018_09_08.tar.gz',
    'deeplab_cityscapes_xception71_trainvalfine':
        'deeplab_cityscapes_xception71_trainvalfine_2018_09_08.tar.gz'
}
_TARBALL_NAME = 'deeplab_model.tar.gz'

model_dir = tempfile.mkdtemp()
tf.io.gfile.makedirs(model_dir)

download_path = os.path.join(model_dir, _TARBALL_NAME)
print('downloading model, this might take a while...')
urllib.request.urlretrieve(_DOWNLOAD_URL_PREFIX + _MODEL_URLS[MODEL_NAME],
                   download_path)
print('download completed! loading DeepLab model...')

MODEL = DeepLabModel(download_path)
print('model loaded successfully!')

downloading model, this might take a while...
download completed! loading DeepLab model...
model loaded successfully!


# パスなどの定数設定

In [33]:
# 解析対象のフォルダパス
path = './photos/2019/'
file_list = os.listdir(path)
# 解析対象のファイル一覧
print('解析対象のファイル一覧:')
for file in file_list:
#     pass
    print(file)

# 結果書き出し用のファイルパス
result_path = './result/2019/vagetation.txt'
f = open(result_path, mode='w')

# print(cv2.imread("./photos/2004/001.JPG"))



解析対象のファイル一覧:
001.JPG
002.JPG
003.JPG
004.JPG
005.JPG
006.JPG
007.JPG
008.JPG
009.JPG
010.JPG
011.JPG
012.JPG
013.JPG
014.JPG
015.JPG
016.JPG
017.JPG
018.JPG
019.JPG
020.JPG
021.JPG
022.JPG
023.JPG
024.JPG
025.JPG
026.JPG
027.JPG
028.JPG
029.JPG
030.JPG
031.JPG
032.JPG
033.JPG
034.JPG
035.JPG
036.JPG
037.JPG
038.JPG


# 画像の解析（ループ処理）

In [34]:
# 結果の書き出し
f = open(result_path, mode='w')
# 処理前の時刻
t1 = time.time()

for file in file_list:
    t3 = time.time()
    
    # 画像の解析（推論の実行）
    print("解析中:", path + file, sep=" ")
    original_im = cv2.imread(path + file)
    pilImg = Image.fromarray(np.uint8(original_im))
    resized_im, seg_map = MODEL.run(pilImg)

    # 緑視率の算出
    flaten_segmap = seg_map.flat
    a = np.array(flaten_segmap)
    # vegetationの割合
    print("緑視率:", (len(np.where(a==FULL_LABEL_MAP[8])[0]) / seg_map.size) *100, "(%)", sep=" ")

    # 結果の可視化（不要ならコメントアウトする）
    # seg_image = label_to_color_image(seg_map).astype(np.uint8)
    # vis_segmentation(resized_im, seg_map)

    f.write("ファイル名（パス）: " + path + file + "\n")
    f.write("緑視率:" + str((len(np.where(a==FULL_LABEL_MAP[8])[0]) / seg_map.size) *100) + "(%)" + "\n")
    
    print("ファイル名（パス）: " + path + file)
    print("緑視率:" + str((len(np.where(a==FULL_LABEL_MAP[8])[0]) / seg_map.size) *100) + "(%)")
    # 処理後の時刻
    t4= time.time()
    elapsed_time_per_case = t4-t3
    print(f"処理時間：{elapsed_time_per_case} 秒")
    print()

f.close()

# 処理後の時刻
t2 = time.time()

elapsed_time = t2-t1
print(f"全件の処理時間：{elapsed_time} 秒")

解析中: ./photos/2019/001.JPG
緑視率: 10.878614359974009 (%)
ファイル名（パス）: ./photos/2019/001.JPG
緑視率:10.878614359974009(%)
処理時間：55.45098114013672 秒

解析中: ./photos/2019/002.JPG
緑視率: 23.500954353476285 (%)
ファイル名（パス）: ./photos/2019/002.JPG
緑視率:23.500954353476285(%)
処理時間：51.45686936378479 秒

解析中: ./photos/2019/003.JPG
緑視率: 31.34695825211176 (%)
ファイル名（パス）: ./photos/2019/003.JPG
緑視率:31.34695825211176(%)
処理時間：50.78728008270264 秒

解析中: ./photos/2019/004.JPG
緑視率: 19.449013157894736 (%)
ファイル名（パス）: ./photos/2019/004.JPG
緑視率:19.449013157894736(%)
処理時間：50.60897874832153 秒

解析中: ./photos/2019/005.JPG
緑視率: 53.71284113060428 (%)
ファイル名（パス）: ./photos/2019/005.JPG
緑視率:53.71284113060428(%)
処理時間：50.581727027893066 秒

解析中: ./photos/2019/006.JPG
緑視率: 34.749634502923975 (%)
ファイル名（パス）: ./photos/2019/006.JPG
緑視率:34.749634502923975(%)
処理時間：50.68494772911072 秒

解析中: ./photos/2019/007.JPG
緑視率: 18.54643843404808 (%)
ファイル名（パス）: ./photos/2019/007.JPG
緑視率:18.54643843404808(%)
処理時間：50.67967128753662 秒

解析中: ./photos/2019/008.JP